In [25]:
pip install pandas gspread pillow requests


Note: you may need to restart the kernel to use updated packages.


from PIL import Image, ImageTk
import tkinter as tk

# Load your certificate template
TEMPLATE_PATH = "certificate_template.png"  # Ensure this file exists
template = Image.open(TEMPLATE_PATH)

# Create a window
root = tk.Tk()
root.title("Click on the Certificate to Set Name Position")

# Convert image to Tkinter format
img = ImageTk.PhotoImage(template)
label = tk.Label(root, image=img)
label.pack()

# Function to get X, Y coordinates on click
def get_position(event):
    print(f"X: {event.x}, Y: {event.y}")  # Print the clicked coordinates
    root.destroy()  # Close window after clicking

# Bind click event
label.bind("<Button-1>", get_position)

# Run the Tkinter loop
root.mainloop()


In [3]:
import pandas as pd
import requests
from PIL import Image, ImageDraw, ImageFont
import os
import re

# Google Sheet CSV link
SHEET_URL = "https://docs.google.com/spreadsheets/d/1Aw2Ne0sDSwpwi4BKRoHlt3ZMGesmWc8PLWD3CcfV1Bw/export?format=csv"

# Certificate template file
TEMPLATE_PATH = "certificate_template.png"  # Ensure this file exists in your directory

# Font file
FONT_PATH = "arial.ttf"  # Change to your actual font file path
FONT_SIZE = 80

# Output folder
OUTPUT_FOLDER = "Generated_Certificates"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Function to format names properly
def format_name(name):
    name = name.strip()  # Remove leading/trailing spaces
    name = re.sub(r"[^a-zA-Z.\s]", "", name)  # Allow letters, spaces, and dots (.)
    name = re.sub(r"\s+", " ", name)  # Replace multiple spaces with a single space
    return name.title()  # Capitalize first letter of each word

# Fetch names from Google Sheets
try:
    df = pd.read_csv(SHEET_URL)
    raw_names = df.iloc[:, 0].dropna().tolist()  # Read first column and remove empty values
    names = [format_name(name) for name in raw_names]  # Apply formatting
    print(f"✅ {len(names)} formatted names fetched from Google Sheets!")
except Exception as e:
    print(f"❌ Error fetching Google Sheet: {e}")
    names = []

# Load the certificate template
try:
    template = Image.open(TEMPLATE_PATH)
    W, H = template.size
except Exception as e:
    print(f"❌ Error loading template: {e}")
    exit()

# Load font
try:
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
except Exception as e:
    print(f"❌ Error loading font: {e}")
    exit()

# 🔹 Define the fixed X, Y position for the name
NAME_POSITION = (742, 670)  # Adjust these values to place text correctly

# Generate certificates for all names
for name in names:
    cert = template.copy()
    draw = ImageDraw.Draw(cert)

    # Auto-center the name horizontally
    bbox = draw.textbbox((0, 0), name, font=font)
    text_width = bbox[2] - bbox[0]
    text_position = ((W - text_width) // 2, NAME_POSITION[1])  # Center horizontally, fixed Y

    # Add the name to the certificate
    text_color = (0, 0, 0)  # Black
    draw.text(text_position, name, fill=text_color, font=font)

    # Save as PDF
    cert_path = os.path.join(OUTPUT_FOLDER, f"{name}.pdf")
    cert.save(cert_path, "PDF")

    print(f"✅ Certificate generated for: {name}")

print("🎉 All certificates generated successfully!")


✅ 4 formatted names fetched from Google Sheets!
✅ Certificate generated for: Amrit
✅ Certificate generated for: Saransh
✅ Certificate generated for: Ritul
✅ Certificate generated for: Amber
🎉 All certificates generated successfully!
